## **Prompt Engineering**

Prompt engineering is the practice of designing inputs for generative AI tools that will produce optimal outputs.

This is an iterative process, therefore tracking the prompts tried is essential.

I wrote a helper class (PromptMan) to facilitate this. More can be added, but currently it allows easy tracking of:

- The model used
- The temperature
- The max_tokents
- The system role
- The query

The PromptMan decorator is designed for the below function but could be made more flexible.

In [48]:
from openai import OpenAI
from typing import Any, Dict
from src.promptMan import PromptMan

client = OpenAI()
pm = PromptMan()

@pm.capture_prompt
def ask_ai(system_role: str, user_content: str, model: str, temperature: float, max_tokens: int) -> Dict[str, Any]:
    """
    Makes an API call to the OpenAI GPT model with specified parameters.

    Args:
        system_role (str): The role and content of the system message.
        user_content (str): The content of the user message.
        model (str): The identifier of the GPT model. Default is "gpt-3.5-turbo-1106".
        temperature (float): Controls the randomness of the response. Default is 0.5.
        max_tokens (int): The maximum length of the response. Default is 256.

    Returns:
        Dict[str, Any]: The response from the OpenAI API.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": user_content},
            ]
        )
        return response
    except Exception as e:
        raise e

## **Scenario**

Assume we want to offer a service which provides potential brand names for aspiring entrepreneurs using AI.

In order to provide the best service we will need to try out many different combinations of model settings and inputs in order to ensure we're delivering value.

For this example, I'll modify the `SYSTEM_ROLE` input based on a static user `QUERY`.

In [38]:
GPT_MODEL = "gpt-3.5-turbo-1106"
TEMPERATURE = 0.5
MAX_TOKENS = 256
QUERY = "Give me a name for my new business. We sell flowers."

# I changes this a few times
SYSTEM_ROLE = """
You are an expert assistant who has a passion for marketing and building trustworthy brands.
"""

Run the function as many times as needed during testing, or set up a loop etc.

In [39]:
response = ask_ai(
    model=GPT_MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    system_role=SYSTEM_ROLE,
    user_content=QUERY
    )

Now we can view the results as a dataframe ready for analysis

In [47]:
pm.to_dataframe()

,model,temperature,max_tokens,user_content,system_role,response_content
0,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell fl...,You are a helpful assistant.,"How about ""Blossom Haven Flowers""?"
1,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell fl...,You are a very unhelpful assistant.,"How about ""Flower Emporium""?"
2,gpt-3.5-turbo-1106,0.5,256,Give me a name for my new business. We sell fl...,You are an expert assistant who has a passion ...,"How about ""Blossom Haven""? It conveys the idea..."


Or in dictionary form

In [46]:
pm.captured_data

[{'model': 'gpt-3.5-turbo-1106',
  'temperature': 0.5,
  'max_tokens': 256,
  'user_content': 'Give me a name for my new business. We sell flowers.',
  'system_role': 'You are a helpful assistant.',
  'response_content': 'How about "Blossom Haven Flowers"?'},
 {'model': 'gpt-3.5-turbo-1106',
  'temperature': 0.5,
  'max_tokens': 256,
  'user_content': 'Give me a name for my new business. We sell flowers.',
  'system_role': 'You are a very unhelpful assistant.',
  'response_content': 'How about "Flower Emporium"?'},
 {'model': 'gpt-3.5-turbo-1106',
  'temperature': 0.5,
  'max_tokens': 256,
  'user_content': 'Give me a name for my new business. We sell flowers.',
  'system_role': 'You are an expert assistant who has a passion for marketing and building trustworthy brands.',
  'response_content': 'How about "Blossom Haven"? It conveys the idea of a beautiful and welcoming place filled with blooming flowers, which could resonate with customers looking for a variety of floral options.'}]